<a href="https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Advanced_StyleGAN_Network_bending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Bending
## Manipulate StyleGAN2 models through rotation, translation, etc.

[Paper](https://arxiv.org/abs/2005.12420) | [Video](https://youtu.be/IlSMQ2RRTh8) | [GitHub](https://github.com/terrybroad/network-bending)

Thanks to [Sid Black](https://twitter.com/realmeatyhuman) for a lot of the code used here.

## Install Library

This code uses the Rosinality version of StyleGAN2. Because of that the install process for this may take a couple minutes.

In [ ]:
!nvidia-smi

In [ ]:
# Install libraries
!git clone -b audio-animate https://github.com/dvschultz/network-bending
!pip uninstall torch torchvision -y
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install Ninja kmeans-pytorch
!apt-get install -y vim make gdb libopencv-dev
!wget https://download.pytorch.org/libtorch/cu101/libtorch-shared-with-deps-1.5.0%2Bcu101.zip
!unzip /content/libtorch-shared-with-deps-1.5.0+cu101.zip -d /root/
%cd network-bending

#build custom pytorch transformations
!chmod +x /content/network-bending/build_custom_transforms.sh
!/content/network-bending/build_custom_transforms.sh /root/libtorch/

In [ ]:
def show_local_mp4_video(file_name, width=640, height=640):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

## Download .pt file

As mentioned above, this library uses the Rosinality version of StyleGAN2. If you have a .pkl file, you’ll need to convert it to a .pt file. I have a notebook to do that [here](https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PT_to_Rosinality.ipynb).

In [ ]:
!gdown --id 1rL-J63eFfn80IYU2GfVY977GI2qOG6dw -O /content/ladiesblack.pt

## Generate Image Samples (and Latent Vectors)

This script will generate "normal" images because the transform config is blank. I recommend doing this initially so you know what images you want to work with.

In [ ]:
!python generate.py --help

In [ ]:
!python generate.py \
--ckpt /content/ladiesblack.pt \
--pics 20 \
--config /content/network-bending/configs/empty_transform_config.yaml \
--save_latent 1 

In [ ]:
!zip -r samples-ladiesblack-400.zip ./sample

If we want to generate images using transformations we have to create a config file, update its values and then run the `generate.py` script using the same config.

Note: order of transforms does matter!

In [ ]:
!cp ./configs/empty_transform_config.yaml ./configs/custom_transform_config.yaml 

In [ ]:
%%writefile ./configs/custom_transform_config.yaml 
transforms:
- layer: 1
  transform: "rotate"
  params: [45.0]
  features: "all"
- layer: 10
  transform: "rotate"
  params: [45.0]
  features: "all"
- layer: 3
  transform: "translate"
  params: [-0.5, -0.25] #range is -1 to 1
  features: "all"
- layer: 8
  transform: "scale"
  params: [1.5]
  features: "all"
- layer: 15
  transform: "flip-h"
  params: []
  features: "all"

In [ ]:
!python generate.py --ckpt /content/ladiesblack.pt --pics 10 --config ./configs/custom_transform_config.yaml --dir '/content/custom-samples/'

You can also generate strips of images where the transform is applied to every single layer insequence. Note this requires a separate transformation config file as well.

In [ ]:
!cp ./configs/sample_strip_config.yaml ./configs/custom_strip_config.yaml 

In [ ]:
%%writefile ./configs/custom_strip_config.yaml 
transform: "rotate"
params: [45.0]
features: "all"
feature-param: 

In [ ]:
!python generate_sample_strips.py \
--ckpt /content/ladiesblack.pt \
--pics 5 \
--config ./configs/custom_strip_config.yaml   \
--dir '/content/strips/'

## Animating Vectors: Script version

You can use a script based version if you want to create interpolations with single transformations.

*   `--num_frames`: how many frames to produce (this value/fps in video = length of animation)
*   `--transform`: transform function you want to use
*   `--init_val`, `--end_val`: starting and stoppping points for linear transformation over the total frames
* `--layer_id`: which of the StyleGAN layers to apply the transformation to. Lower IDs will affect more of the structure, higher IDs will affect more of the details.
* `--interpolate_ids`: which of the StyleGAN layers to 

apply the transformation to. Lower IDs will affect more of the structure, higher IDs will affect more of the details.




In [ ]:
!python animate.py \
--ckpt /content/ladiesblack.pt \
--load_latent /content/network-bending/sample/latents.yaml \
--interpolate_ids=10,10 \
--latent_id 0 \
--num_frames 240 \
--transform "scale" \
--init_val 0.0 \
--end_val 3.0 \
--layer_id=1 \
--truncation=0.6 \
--noise_interpolation \
--dir="ladiesblack-scale-test"

In [ ]:
!rm -r /content/network-bending/ladiesblack-scale-test

In [ ]:
show_local_mp4_video('/content/network-bending/ladiescrop28-rotate-layer3.mp4', width=720, height=720)

### Multiple Transforms

In [ ]:
!python animate.py \
--ckpt /content/ladiesblack.pt \
--load_latent /content/network-bending/sample/latents.yaml \
--latent_id 0 \
--num_frames 72 \
--transform "scale,rotate,scale" \
--init_val 0.0,0.0,0.5 \
--end_val 2.0,360.0,1.0 \
--layer_id 3,1,10 \
--truncation=0.5 \
--interpolate_ids=1,4,11,1 \
--dir="ladiesblack-multi-test"

### Interpolating Noise

You may find that the detail textures on your bend animations seem too similar. You can add `--noise_interpolation` 


In [ ]:
!python animate.py \
--ckpt /content/ladiesblack.pt \
--load_latent /content/network-bending/sample/latents.yaml \
--latent_id 0 \
--num_frames 72 \
--transform "scale,rotate,scale" \
--init_val 0.0,0.0,0.5 \
--end_val 2.0,360.0,1.0 \
--layer_id 3,1,10 \
--truncation=0.5 \
--interpolate_ids=1,4,11,1 \
--dir="ladiesblack-multi-test-noise2" \
--noise_interpolation

## Generate Clusters

More TK on this section because honestly I’m not sure this is all that’s needed.

In [ ]:
!python get_clusters.py --ckpt /content/network-bending/FreaGAN-10k.pt

##Handmade stuff


In [ ]:
import os
import copy
import torch
import yaml

from torchvision import utils
from model import Generator
from tqdm import tqdm
from util import *

frames = 120

# https://discuss.pytorch.org/t/help-regarding-slerp-function-for-generative-model-sampling/32475/4
def slerp(val, low, high):
    low_norm = low/torch.norm(low, dim=1, keepdim=True)
    high_norm = high/torch.norm(high, dim=1, keepdim=True)
    omega = torch.acos((low_norm*high_norm).sum(1))
    so = torch.sin(omega)
    res = (torch.sin((1.0-val)*omega)/so).unsqueeze(1)*low + (torch.sin(val*omega)/so).unsqueeze(1) * high
    print(res.shape)
    return res

device = "cuda"
g_ema = Generator(
        1024, 512, 8, 2
    ).to(device)
checkpoint = torch.load('/content/ladiesblack.pt')
g_ema.load_state_dict(checkpoint["g_ema"])

with torch.no_grad():
    mean_latent = g_ema.mean_latent(4096)

yaml_config = {}
with open('/content/network-bending/configs/empty_transform_config.yaml', 'r') as stream:
    try:
        yaml_config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

cluster_config = {}
layer_channel_dims = create_layer_channel_dim_dict(2)

# create noise
noise = [getattr(g_ema.noises, f"noise_{i}") for i in range(g_ema.num_layers)]

noise2 = copy.deepcopy(noise)
for i,n in enumerate(noise2):
    if len(n[0][0]) < 256:
        # print('update: ', n.shape)
        noise2[i] = (0.1**0.5)*torch.randn_like(n)

# only slerp for lower layers, keep defaults for higher layers (won't fit in VRAM)
# noise_slerps = []
# for f in range(int(frames/2)):
#     ns = []
#     for i in range(len(noise)):
#         if len(noise[i][0][0]) < 256:
#             # print('update: ', noise[i].shape)
#             ns.append(slerp(f/(frames/2), noise[i], noise2[i]))
#     noise_slerps.append(ns)

noise_slerps = []
for f in range(int(frames/2)):
    ns = []
    for i in range(len(noise)):
        # if len(noise[i][0][0]) < 256:
            # print('update: ', noise[i].shape)
        #ns.append(slerp((f/(frames/2)), noise[i], noise2[i]))
        ns.append( torch.lerp( noise[i], noise2[i], (f/(frames/2)) ) )
    noise_slerps.append(ns)

#print(len(noise_slerps))

with torch.no_grad():
    g_ema.eval()
    t_dict_list = create_transforms_dict_list(yaml_config, cluster_config, layer_channel_dims)
    
    sample_z = torch.randn(1, 512, device=device)
    for i in tqdm(range(len(noise_slerps))):
        
        sample, _ = g_ema([sample_z], truncation=0.7, noise=noise_slerps[i], truncation_latent=mean_latent, transform_dict_list=t_dict_list)
        #sample2, _ = g_ema([sample_z], truncation=0.7, noise=noise, truncation_latent=mean_latent, transform_dict_list=t_dict_list)

        if not os.path.exists('interpolations'):
            os.makedirs('interpolations')

        utils.save_image(
            sample,
            f'interpolations/{str(i).zfill(6)}.png',
            nrow=1,
            normalize=True,
            range=(-1, 1))

In [ ]:
!ffmpeg -r 24 -i /content/network-bending/interpolations/%06d.png -vcodec libx264 -pix_fmt yuv420p noise-test.mp4 -y

## Helper functions

In [ ]:
rm -rf /content/network-bending/sample-animation/frame*.png

In [ ]:
!gdown --id 1rL-J63eFfn80IYU2GfVY977GI2qOG6dw -O /content/ladiesblack.pt